In [255]:
import numpy as np 
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,random_split,Dataset
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [256]:
# train = pd.read_csv('C:/Users/MJ/Desktop/FunctionApproximation/train.csv')
# test =pd.read_csv('C:/Users/MJ/Desktop/FunctionApproximation/test.csv')
# val_size = int(0.2 * len(train))
# train_size=len(train)-val_size
# train, valid = random_split(train,[train_size, val_size])

In [257]:
cuda = True if torch.cuda.is_available() else False

In [258]:
train=pd.read_csv("C:/Users/MJ/Desktop/FunctionApproximation/train.csv")
test=pd.read_csv("C:/Users/MJ/Desktop/FunctionApproximation/test.csv")

In [259]:
class FADataset(Dataset):
    def __init__(self, data,train=True):
        super().__init__()
        if(train):
            self.x=data.drop(['id','y'],axis=1).values
            self.x=torch.tensor(self.x,dtype=torch.float32)
            self.y=torch.tensor(data['y'].values,dtype=torch.float32)
        else:
            self.x=data.drop(['id'],axis=1).values
            self.x=torch.tensor(self.x,dtype=torch.float32)
    def __len__(self):
        return len(self.x)
    def __getitem__(self, idx):
        data=self.x[idx]
        target=self.y[idx]
        return data,target,idx

    # def update_loss_value(self, idx):
    #     self.flag[idx] = True
        

In [260]:
batch_size=80
dataset=FADataset(train,train=True)
trainset, validset = random_split(dataset, [0.9, 0.1])
train_dl= DataLoader(trainset, shuffle=True, batch_size=batch_size)
valid_dl= DataLoader(validset, shuffle=True, batch_size=batch_size)
testset=FADataset(test,train=False)
test_dl=DataLoader(testset,shuffle=False, batch_size=batch_size)

In [261]:
class MLP(nn.Module):
    def __init__(self, input_dim, num_hidden1, num_hidden2,num_hidden3,num_hidden4,num_hidden5,num_hidden6, output_dim):
        super().__init__()
        self.model=nn.Sequential(nn.Linear(input_dim,num_hidden1),
                                nn.ReLU(True),
                                nn.Linear(num_hidden1,num_hidden2),
                                nn.ReLU(True),
                                nn.Linear(num_hidden2,num_hidden3),
                                nn.ReLU(True),
                                nn.Linear(num_hidden3,num_hidden4),
                                nn.ReLU(True),  
                                nn.Linear(num_hidden4,num_hidden5),
                                nn.ReLU(True),  
                                nn.Linear(num_hidden5,num_hidden6),
                                nn.Linear(num_hidden6,output_dim)
                                )
    def forward(self, x):
        return self.model(x)


In [262]:
model= MLP(input_dim=2, num_hidden1=80,num_hidden2=128,num_hidden3=256,num_hidden4=512,num_hidden5=256,num_hidden6=128, output_dim=1)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [263]:
num_epochs = 50
losses=[]
size = len(train_dl)
model.train()
for epoch in range(num_epochs):
    losses=[]
    for batch,(data, target,idx) in enumerate(train_dl):
        outputs = model(data)
        loss = criterion(outputs.reshape(-1), target.reshape(-1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch % 80 == 0: #每通過batch個數量的資料就輸出一次損失的值
            # print(f"loss: {loss.item():>7f} ")
            losses.append(loss.item())
        
    print("lossmean=:",sum(losses)/len(losses))

lossmean=: 0.8347115144133568
lossmean=: 0.1383598893880844
lossmean=: 0.1128661036491394
lossmean=: 0.0947590321302414
lossmean=: 0.08899831771850586
lossmean=: 0.07309763133525848
lossmean=: 0.06389287114143372
lossmean=: 0.055696140974760056
lossmean=: 0.0636394526809454
lossmean=: 0.050134479999542236
lossmean=: 0.04985796473920345
lossmean=: 0.037437865510582924
lossmean=: 0.03836698643863201
lossmean=: 0.0347941629588604
lossmean=: 0.0373261496424675
lossmean=: 0.032818255946040154
lossmean=: 0.025913610123097897
lossmean=: 0.02568631526082754
lossmean=: 0.025903836823999882
lossmean=: 0.028533288277685642
lossmean=: 0.026719513349235058
lossmean=: 0.02163355704396963
lossmean=: 0.024305399507284164
lossmean=: 0.020626570098102093
lossmean=: 0.02250686101615429
lossmean=: 0.022948212921619415
lossmean=: 0.023500023409724236
lossmean=: 0.02962900511920452
lossmean=: 0.024226967245340347
lossmean=: 0.023135513998568058
lossmean=: 0.024100403301417828
lossmean=: 0.03323686495423317


In [264]:
min(losses)

0.025691401213407516

In [265]:
model.eval()
losses=[]
for batch,(data, target,idx) in enumerate(valid_dl):
    outputs = model(data)
    loss = criterion(outputs.reshape(-1), target.reshape(-1))
    losses.append(loss.item())
print(f"loss: {sum(losses)/len(losses):>7f} ")

loss: 0.023640 


In [266]:
model.eval()
output=[]
for i in range(len(testset)):
    output.append((model(testset.x[i])).item())

In [267]:
min(output)

0.2990783452987671

In [268]:
sample_submission=pd.read_csv("C:/Users/MJ/Desktop/FunctionApproximation/sample_submission.csv")
sample_submission['y']=output
sample_submission.to_csv("C:/Users/MJ/Desktop/sample_submission.csv",index=False)

In [269]:
import optuna

In [270]:
def build_model(params):
    
    input_dim = 2
    output_dim=1
    return nn.Sequential(
                            nn.Linear(input_dim,params['num_hidden1']),
                            nn.ReLU(True),
                            nn.Linear(params['num_hidden1'],params['num_hidden2']),
                            nn.ReLU(True),
                            nn.Linear(params['num_hidden2'],params['num_hidden3']),
                            nn.ReLU(True),
                            nn.Linear(params['num_hidden3'],params['num_hidden4']),
                            nn.ReLU(True),  
                            nn.Linear(params['num_hidden4'],params['num_hidden5']),
                            nn.ReLU(True),  
                            nn.Linear(params['num_hidden5'],params['num_hidden6']),
                            nn.Linear(params['num_hidden6'],output_dim)
                        )

In [271]:
def train_and_evaluate(params,model):

    optimizer = getattr(torch.optim, params['optimizer'])(model.parameters(), lr= params['learning_rate'])
    

    dataset=FADataset(train,train=True)
    trainset, validset = random_split(dataset, [0.9, 0.1])
    train_dl= DataLoader(trainset, shuffle=True, batch_size=params['batch_size'])
    valid_dl= DataLoader(validset, shuffle=True, batch_size=params['batch_size'])
    
    criterion = nn.MSELoss()

    losses=[]
    num_epochs=50
    model.train()
    for epoch in range(num_epochs):
        for batch,(data, target,idx) in enumerate(train_dl):
            outputs = model(data)
            loss = criterion(outputs.reshape(-1), target.reshape(-1))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


    model.eval()
    losses=[]
    for batch,(data, target,idx) in enumerate(valid_dl):
        outputs = model(data)
        loss = criterion(outputs.reshape(-1), target.reshape(-1))
        losses.append(loss.item())
    print(f"loss: {sum(losses)/len(losses):>7f} ")
    loss=sum(losses)/len(losses)
    
    return loss


In [272]:

def objective(trial):

    params = {
            'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1e-1),
            'optimizer': trial.suggest_categorical("optimizer", ["Adam", "SGD"]),
            'num_hidden1': trial.suggest_int("num_hidden1", 4, 500),
            'num_hidden2': trial.suggest_int("num_hidden2", 4, 600),
            'num_hidden3': trial.suggest_int("num_hidden3", 4, 600),
            'num_hidden4': trial.suggest_int("num_hidden4", 4, 600),
            'num_hidden5': trial.suggest_int("num_hidden5", 4, 600),
            'num_hidden6': trial.suggest_int("num_hidden6", 4, 600),
            'batch_size': trial.suggest_int("batch_size", 64, 192,step=16)
            }
    
    
    model= build_model(params)
    loss=train_and_evaluate(params,model)
        
    
    return loss

In [273]:
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=30)

[I 2023-04-29 20:34:09,532] A new study created in memory with name: no-name-e3d74531-0a02-4ffe-96e1-5304cdf7338d
[I 2023-04-29 20:34:16,204] Trial 0 finished with value: 0.10774971097707749 and parameters: {'learning_rate': 0.01815253213308689, 'optimizer': 'SGD', 'num_hidden1': 313, 'num_hidden2': 107, 'num_hidden3': 148, 'num_hidden4': 174, 'num_hidden5': 176, 'num_hidden6': 323, 'batch_size': 160}. Best is trial 0 with value: 0.10774971097707749.


loss: 0.107750 


[I 2023-04-29 20:34:32,957] Trial 1 finished with value: 0.22592079937458037 and parameters: {'learning_rate': 0.08377454390492545, 'optimizer': 'Adam', 'num_hidden1': 498, 'num_hidden2': 69, 'num_hidden3': 578, 'num_hidden4': 214, 'num_hidden5': 570, 'num_hidden6': 303, 'batch_size': 192}. Best is trial 0 with value: 0.10774971097707749.


loss: 0.225921 


[I 2023-04-29 20:34:58,440] Trial 2 finished with value: 0.1419107038527727 and parameters: {'learning_rate': 0.059293498092321574, 'optimizer': 'Adam', 'num_hidden1': 97, 'num_hidden2': 46, 'num_hidden3': 545, 'num_hidden4': 564, 'num_hidden5': 93, 'num_hidden6': 425, 'batch_size': 112}. Best is trial 0 with value: 0.10774971097707749.


...

loss: 0.042797 


[I 2023-04-29 20:43:38,219] Trial 28 finished with value: 0.03129450045526028 and parameters: {'learning_rate': 0.033321594217801793, 'optimizer': 'SGD', 'num_hidden1': 9, 'num_hidden2': 553, 'num_hidden3': 416, 'num_hidden4': 403, 'num_hidden5': 108, 'num_hidden6': 206, 'batch_size': 64}. Best is trial 12 with value: 0.025936122185417583.


loss: 0.031295 


[I 2023-04-29 20:43:48,247] Trial 29 finished with value: 0.09745743721723557 and parameters: {'learning_rate': 0.018824891459769844, 'optimizer': 'SGD', 'num_hidden1': 457, 'num_hidden2': 300, 'num_hidden3': 481, 'num_hidden4': 168, 'num_hidden5': 194, 'num_hidden6': 319, 'batch_size': 160}. Best is trial 12 with value: 0.025936122185417583.


loss: 0.097457 


In [282]:
study.best_params

{'learning_rate': 0.006759886458677327,
 'optimizer': 'Adam',
 'num_hidden1': 400,
 'num_hidden2': 431,
 'num_hidden3': 428,
 'num_hidden4': 372,
 'num_hidden5': 103,
 'num_hidden6': 175,
 'batch_size': 128}

In [283]:
study.best_value

0.025936122185417583

In [284]:
display(optuna.visualization.plot_param_importances(study))
display(optuna.visualization.plot_parallel_coordinate(study))
display(optuna.visualization.plot_optimization_history(study))

In [285]:
def best(params):
 
    model= build_model(params)
    optimizer = getattr(torch.optim, params['optimizer'])(model.parameters(), lr= params['learning_rate'])
    

    dataset=FADataset(train,train=True)
    trainset, validset = random_split(dataset, [0.9, 0.1])
    train_dl= DataLoader(trainset, shuffle=True, batch_size=params['batch_size'])
    valid_dl= DataLoader(validset, shuffle=True, batch_size=params['batch_size'])

   
    criterion = nn.MSELoss()

    losses=[]
    size = len(train_dl)
    model.train()
    for epoch in range(num_epochs):
        for batch,(data, target,idx) in enumerate(train_dl):
            outputs = model(data)
            loss = criterion(outputs.reshape(-1), target.reshape(-1))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


    model.eval()
    losses=[]
    for batch,(data, target,idx) in enumerate(valid_dl):
        outputs = model(data)
        loss = criterion(outputs.reshape(-1), target.reshape(-1))
        losses.append(loss.item())
    print(f"loss: {sum(losses)/len(losses):>7f} ")
    loss=sum(losses)/len(losses)
    return model

In [287]:
study.best_params['optimizer']

'Adam'

In [291]:
model=best(study.best_params)

loss: 0.039615 


In [289]:
model.eval()
output=[]
for i in range(len(testset)):
    output.append((model(testset.x[i])).item())

In [290]:
sample_submission=pd.read_csv("C:/Users/MJ/Desktop/FunctionApproximation/sample_submission.csv")
sample_submission['y']=output
sample_submission.to_csv("C:/Users/MJ/Desktop/sample_submission.csv",index=False)